GUI Code

Used tech : Pyqt5

In [ ]:
# importing librairies
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QTextEdit, QLineEdit, QLabel, QComboBox, QFrame, QDialog
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas

# Creating class
class DatabaseApp(QWidget):
    def __init__(self):
        super(DatabaseApp, self).__init__()

        self.connection = None
        self.cursor = None
        self.emp_id = None  # Make emp_id an instance variable

        self.init_ui()

    def init_ui(self):
        # Widgets
        self.connection_label = QLabel("Database Connection: Not Connected")
        self.query_input = QLineEdit()
        self.query_results = QTextEdit()
        self.query_results.setReadOnly(True)

        self.operation_label = QLabel("Choose Operation:") # Dropdown Label
        self.operation_combo = QComboBox()
        self.operation_combo.addItems(["Connect", "Insert", "Delete", "Update", "Run Query", "Get Payroll", "Get Attendance", "Close Connection", "Exit"]) # Dropdown Options

        self.execute_button = QPushButton("Execute")
        self.execute_button.clicked.connect(self.execute_operation)

        # Layout
        vbox = QVBoxLayout()
        vbox.addWidget(self.connection_label)
        vbox.addWidget(self.operation_label)
        vbox.addWidget(self.operation_combo)
        vbox.addWidget(self.query_input)
        vbox.addWidget(self.query_results)
        vbox.addWidget(self.execute_button)

        self.setGeometry(100, 100, 800, 600)    # Dimensions of the GUI Window
        self.setWindowTitle("Payroll Management System")

        self.setLayout(vbox)

    # Connect to the database using mysql connector
    def db_connect(self):
        try:
            self.connection = mysql.connector.connect(
                host="127.0.0.1",
                user="root",                # User name from the connection in mysql DB
                # if db has the password then remove comment for the below code
                # my_password = getpass()
                # password=my_password
                database='payroll'          # Select which DB/Schema you want to use.
            )

            if self.connection.is_connected():
                self.connection_label.setText("Database Connection: Connected")
                return True
            else:
                self.connection_label.setText("Database Connection: Failed")
                return False

        except mysql.connector.Error as e:
            self.connection_label.setText(f"Error connecting to database: {e}")
            return False

    # Execution of function , based on the selected input from dropdown.
    def execute_operation(self):
        operation = self.operation_combo.currentText()

        if operation == "Connect":
            self.db_connect()
        elif operation == "Insert":
            self.execute_insert()
        elif operation == "Delete":
            self.execute_delete()
        elif operation == "Update":
            self.execute_update()
        elif operation == "Run Query":
            self.execute_select()
        elif operation == "Get Payroll":
            self.get_payroll()
        elif operation == "Get Attendance":
            self.get_attendance()
        elif operation == "Close Connection":
            self.close_connection()
        elif operation == "Exit":
            self.close_app()

    # Insert Operation
    def execute_insert(self):
        if self.connection and self.connection.is_connected():
            try:
                sql = self.query_input.text()
                cursor = self.connection.cursor()
                cursor.execute(sql)
                self.connection.commit()
                self.refresh_query_results()
                self.query_results.append(f"{cursor.rowcount} row(s) successfully inserted.")

            # Print the error thrown by mysql connector on GUI
            except Exception as e:
                self.query_results.append(f"Error inserting data: {e}")

    # Delete Operation
    def execute_delete(self):
        if self.connection and self.connection.is_connected():
            try:
                sql = self.query_input.text()
                cursor = self.connection.cursor()
                cursor.execute(sql)
                self.connection.commit()
                self.refresh_query_results()
                self.query_results.append(f"{cursor.rowcount} row(s) successfully deleted.")

            # Print the error thrown by mysql connector on GUI
            except Exception as e:
                self.query_results.append(f"Error deleting data: {e}")

    # Update Operation
    def execute_update(self):
        if self.connection and self.connection.is_connected():
            try:
                sql = self.query_input.text()
                cursor = self.connection.cursor()
                cursor.execute(sql)
                self.connection.commit()
                self.refresh_query_results()
                self.query_results.append(f"{cursor.rowcount} row(s) successfully updated.")

            # Print the error thrown by mysql connector on GUI
            except Exception as e:
                self.query_results.append(f"Error updating data: {e}")

     # Select Operation
    def execute_select(self):
        if self.connection and self.connection.is_connected():
            try:
                sql = self.query_input.text()
                cursor = self.connection.cursor()
                cursor.execute(sql)
                results = cursor.fetchall()
                self.refresh_query_results()
                for row in results:
                    self.query_results.append(str(row))

            # Print the error thrown by mysql connector on GUI
            except Exception as e:
                self.query_results.append(f"Error executing query: {e}")

    # This function will give the payroll history of the employee for emp_ID give in input
    def get_payroll(self):
        if self.connection and self.connection.is_connected():
            try:
                self.emp_id = self.query_input.text()
                cursor = self.connection.cursor(dictionary=True)
                query = f"SELECT * FROM payroll_ WHERE emp_ID = '{self.emp_id}' ORDER BY month_ "
                cursor.execute(query)
                results = cursor.fetchall()

                self.refresh_query_results()
                if results:
                    for row in results:
                        self.query_results.append(f"month_: {row['month_']}, year_: {row['year_']}, gross_pay: {row['gross_pay']},net_pay: {row['net_pay']}")
                else:
                    self.query_results.append("No payment records found.")


            # Print the error thrown by mysql connector on GUI
            except Exception as e:
                self.query_results.append(f"Error fetching payroll data: {e}")

        else:
            self.query_results.append("Database connection failed.")

    # This function will give the Attendance history of the employee for emp_ID give in input
    def get_attendance(self):
        if self.connection and self.connection.is_connected():
            try:
                emp_id = self.query_input.text()
                cursor = self.connection.cursor(dictionary=True)
                query = f"SELECT * FROM attendance WHERE emp_ID = '{emp_id}'"
                cursor.execute(query)
                results = cursor.fetchall()

                if results: # Conversion of results to Dataframe , If exist.
                    df = pd.DataFrame(results)
                    df['date'] = pd.to_datetime(df['year_'].astype(str) + '-' + df['month_'].astype(str), format='%Y-%m')
                    df = df.sort_values('date')

                    # separate window
                    self.plot_attendance_graph(df)

                else:
                    self.query_results.append(f"No attendance records found for Employee {emp_id}.")

            # Print the error thrown by mysql connector on GUI
            except Exception as e:
                self.query_results.append(f"Error fetching attendance data: {e}")



        else:
            self.query_results.append("Database connection failed.")

    # Close DB Connection
    def close_connection(self):
        if self.connection and self.connection.is_connected():
            try:
                self.connection.close()
                self.connection_label.setText("Database Connection: Closed")

            # Print the error thrown by mysql connector on GUI
            except Exception as e:
                self.query_results.append(f"Error closing connection: {e}")

    # Exit GUI
    def close_app(self):
        if self.connection and self.connection.is_connected():
            self.close_connection()
        print("Exiting the program.")
        sys.exit()

    # refresh window before printing new results
    def refresh_query_results(self):
        self.query_results.clear()

    # Plot attendance for any emp_ID by converting results into DF and use matplotlib library to plot.
    def plot_attendance_graph(self, df):
        dialog = QDialog(self)
        dialog.setWindowTitle("Attendance Graph")

        canvas = FigureCanvas(plt.Figure(figsize=(10, 6)))
        dialog_layout = QVBoxLayout()
        dialog_layout.addWidget(canvas)

        dialog.setLayout(dialog_layout)

        ax = canvas.figure.add_subplot(111)
        ax.plot(df['date'], df['days_worked'], marker='o', linestyle='-', color='b')
        ax.set_title("Employee Attendance")
        ax.set_xlabel("Month and Year")
        ax.set_ylabel("Days Worked")
        ax.grid(True)

        dialog.exec_()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = DatabaseApp()
    window.setWindowTitle("Payroll Management System") # Title for the GUI
    window.show()
    sys.exit(app.exec_())
